In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
def dividir_rango_fechas(desde, hasta):
    """Divide un rango de fechas en segmentos de hasta un año."""
    inicio = []
    final = []
    fecha_actual = desde
    while fecha_actual < hasta:
        siguiente = min(fecha_actual + timedelta(days=365), hasta)
        inicio.append(fecha_actual)
        final.append(siguiente)
        fecha_actual = siguiente + timedelta(days=1)
    return inicio, final


In [11]:
principales_variables = pd.read_csv('principalesVariables.csv')
idvariables = principales_variables['idVariable'].tolist() 
fecha_inicio = datetime(1996,1,1)
fecha_final = datetime.now() - timedelta(days=datetime.now().weekday() + 2 if datetime.now().weekday() < 5 else 1)
inicio, final = dividir_rango_fechas(fecha_inicio, fecha_final)
empty_df = pd.DataFrame()
df_dicc={}
fechas = pd.date_range(start=fecha_inicio, end=fecha_final, freq='D')
registros_bcra = pd.DataFrame({'fecha': fechas})

In [12]:
for v in idvariables:
    for i, f in zip(inicio, final):
        url = f"https://api.bcra.gob.ar/estadisticas/v2.0/datosvariable/{v}/{i.strftime('%Y-%m-%d')}/{f.strftime('%Y-%m-%d')}"
        try:
            response = requests.get(url, verify=False)

            # Convertir la respuesta JSON a un objeto de Python
            data = response.json()
        
            results = data.get("results")
            if results:
                print(f'Cargando datos {i} a {f}...')
                # Crear un DataFrame temporal con los resultados
                temp_df = pd.DataFrame(results)
                # Concatenar el DataFrame temporal con el acumulador
                empty_df = pd.concat([empty_df, temp_df])
            else:
                print(f'No hay datos en rango {i} a {f}')

        except KeyError as e:
            print(f"Error procesando los datos para el rango {i} - {f}: {e}")

    df_dicc[v] = empty_df
    empty_df = empty_df.iloc[0:0]
    print('='*100)
    print(f'Variable {v} cargada.')
    print('='*100)
    print('='*100)

    

Cargando datos 1996-01-01 00:00:00 a 1996-12-31 00:00:00...
Cargando datos 1997-01-01 00:00:00 a 1998-01-01 00:00:00...
Cargando datos 1998-01-02 00:00:00 a 1999-01-02 00:00:00...
Cargando datos 1999-01-03 00:00:00 a 2000-01-03 00:00:00...
Cargando datos 2000-01-04 00:00:00 a 2001-01-03 00:00:00...
Cargando datos 2001-01-04 00:00:00 a 2002-01-04 00:00:00...
Cargando datos 2002-01-05 00:00:00 a 2003-01-05 00:00:00...
Cargando datos 2003-01-06 00:00:00 a 2004-01-06 00:00:00...
Cargando datos 2004-01-07 00:00:00 a 2005-01-06 00:00:00...
Cargando datos 2005-01-07 00:00:00 a 2006-01-07 00:00:00...
Cargando datos 2006-01-08 00:00:00 a 2007-01-08 00:00:00...
Cargando datos 2007-01-09 00:00:00 a 2008-01-09 00:00:00...
Cargando datos 2008-01-10 00:00:00 a 2009-01-09 00:00:00...
Cargando datos 2009-01-10 00:00:00 a 2010-01-10 00:00:00...
Cargando datos 2010-01-11 00:00:00 a 2011-01-11 00:00:00...
Cargando datos 2011-01-12 00:00:00 a 2012-01-12 00:00:00...
Cargando datos 2012-01-13 00:00:00 a 201

In [13]:
for clave, df in df_dicc.items():
    df['fecha'] = pd.to_datetime(df['fecha'])
    # Combinar los datos basados en la columna 'fecha'
    registros_bcra = registros_bcra.merge(
        df[['fecha', 'valor']],
        on='fecha',
        how='left',
        suffixes=('', f'_{clave}')  # Evita conflictos en los nombres
    )
    # Renombrar la columna 'valor' resultante al nombre de la clave
    registros_bcra.rename(columns={'valor': clave}, inplace=True)

# Mostrar el resultado
print(registros_bcra)

           fecha        1        4        5     6        7        8     9  10  \
0     1996-01-01      NaN      NaN      NaN   NaN      NaN      NaN   NaN NaN   
1     1996-01-02  17314.0      NaN      NaN   NaN      NaN      NaN   NaN NaN   
2     1996-01-03  17404.0      NaN      NaN   NaN      NaN      NaN   NaN NaN   
3     1996-01-04  17435.0      NaN      NaN   NaN      NaN      NaN   NaN NaN   
4     1996-01-05  17609.0      NaN      NaN   NaN      NaN      NaN   NaN NaN   
...          ...      ...      ...      ...   ...      ...      ...   ...  ..   
10557 2024-11-26  30928.0  1037.67  1007.75  35.0  37.3125  37.9375  40.0 NaN   
10558 2024-11-27  31484.0  1036.96  1008.75  35.0  36.3750  37.5625  40.0 NaN   
10559 2024-11-28  31637.0  1037.24  1009.25  35.0  37.0000  37.8750  40.0 NaN   
10560 2024-11-29  30214.0  1040.27  1011.75  35.0  36.5000  37.5000  40.0 NaN   
10561 2024-11-30      NaN      NaN      NaN   NaN      NaN      NaN   NaN NaN   

          11  ...  29      

In [14]:
# Rellenar valores nulos con interpolacion lineal
for col in idvariables:
    if col in registros_bcra.columns:
        # Asegurarnos de que hay al menos un valor no nulo antes y después
        if registros_bcra[col].notna().sum() > 1:
            # Rellenar valores nulos con interpolación lineal
            registros_bcra[col] = registros_bcra[col].interpolate(method='linear', limit_direction='forward')

print(registros_bcra)


           fecha        1        4        5     6        7        8     9  \
0     1996-01-01      NaN      NaN      NaN   NaN      NaN      NaN   NaN   
1     1996-01-02  17314.0      NaN      NaN   NaN      NaN      NaN   NaN   
2     1996-01-03  17404.0      NaN      NaN   NaN      NaN      NaN   NaN   
3     1996-01-04  17435.0      NaN      NaN   NaN      NaN      NaN   NaN   
4     1996-01-05  17609.0      NaN      NaN   NaN      NaN      NaN   NaN   
...          ...      ...      ...      ...   ...      ...      ...   ...   
10557 2024-11-26  30928.0  1037.67  1007.75  35.0  37.3125  37.9375  40.0   
10558 2024-11-27  31484.0  1036.96  1008.75  35.0  36.3750  37.5625  40.0   
10559 2024-11-28  31637.0  1037.24  1009.25  35.0  37.0000  37.8750  40.0   
10560 2024-11-29  30214.0  1040.27  1011.75  35.0  36.5000  37.5000  40.0   
10561 2024-11-30  30214.0  1040.27  1011.75  35.0  36.5000  37.5000  40.0   

         10     11  ...    29        30       31      32     34     35     

In [ ]:
# Calcular tabla de metricas moviles segun 'xdays'

# Asegúrate de que 'fecha' esté en formato datetime
registros_bcra['fecha'] = pd.to_datetime(registros_bcra['fecha'])


xdays = 30  # Número de días para la ventana móvil

# Crear un DataFrame para almacenar las métricas
df_metrics = pd.DataFrame({'fecha': registros_bcra['fecha']})
registros_bcra = registros_bcra.set_index('fecha')

for columna in idvariables:
    if columna in registros_bcra.columns:
        registros_bcra = registros_bcra.sort_values('fecha')  # Asegurar el orden cronológico
        registros_bcra[columna] = registros_bcra[columna].astype(float)

        # Promedio móvil
        df_metrics[f"{columna}_promedio_movil"] = (
            registros_bcra.rolling(f"{xdays}D", on='fecha')[columna].mean()
        )

        # Desviación estándar
        desviacion_estandar = registros_bcra.rolling(f"{xdays}D", on='fecha')[columna].std()

        # Calcular la desviación estándar relativa como porcentaje de la media
        desviacion_estandar_relativa = (desviacion_estandar / registros_bcra.rolling(f"{xdays}D", on='fecha')[columna].mean()) * 100
        
        df_metrics[f"{columna}_desviacion_estandar_relativa"] = desviacion_estandar_relativa

        # Promedio de la desviación estándar relativa (promedio móvil de la desviación estándar relativa)
        df_metrics[f"{columna}_promedio_desviacion_estandar_relativa"] = (
            desviacion_estandar_relativa.rolling(f"{xdays}D", on='fecha').mean()
        )

print(df_metrics)

ValueError: invalid on specified as fecha, must be a column (of DataFrame), an Index or None

In [15]:
registros_bcra.to_csv('Registros BCRA desde 1996.csv', index=False)